In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2 

In [ ]:
import os
import datajoint as dj
dj.config['database.host'] = os.environ['DJ_HOST']
dj.config['database.user'] = os.environ['DJ_USER']
dj.config['database.password'] = os.environ['DJ_PASS']
dj.config['enable_python_native_blobs'] = True

name = 'realdata' #"simdata"
dj.config['schema_name'] = f"konstantin_nnsysident_{name}"

In [ ]:
import torch
import numpy as np
import pickle 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
import matplotlib.pyplot as plt
import re
import seaborn as sns

import nnfabrik
from nnfabrik.main import *
from nnfabrik import builder
from nnfabrik.utility.hypersearch import Bayesian

from nnsysident.tables.experiments import *
from nnsysident.tables.bayesian import *
from nnsysident.datasets.mouse_loaders import static_shared_loaders
from nnsysident.datasets.mouse_loaders import static_loaders
from nnsysident.datasets.mouse_loaders import static_loader

def find_number(text, c):
    return re.findall(r'%s(\d+)' % c, text)

def get_transfer():
    # prepare the Transfer table in a way that all the info about the transferred model is in the DataFrame. Just pd.merge (on transfer_fn and transfer_hash)
    # it then with the model that the transferred model was used for. 
    transfer = pd.DataFrame(Transfer.fetch())
    transfer = pd.concat([transfer, transfer['transfer_config'].apply(pd.Series)], axis = 1).drop('transfer_config', axis = 1)

    tm = pd.DataFrame((TrainedModel * Dataset).fetch()).rename(
        columns = {'model_hash': 't_model_hash', 'trainer_hash': 't_trainer_hash', 'dataset_hash': 't_dataset_hash'})

    transfer = pd.merge(transfer, tm, how='inner', on=['t_model_hash', 't_trainer_hash', 't_dataset_hash'])
    transfer = pd.concat([transfer, transfer['dataset_config'].apply(pd.Series)], axis = 1).drop('dataset_config', axis = 1)
    transfer.columns = ['t_' + col if col[:2] != 't_' and col[:8] != 'transfer'  else col for col in transfer.columns]
    return transfer

In [ ]:
SeedBayesian.insert1(dict(seed=1))

In [ ]:
Fabrikant()

In [ ]:
Trainer()

# Add the Dataset

In [ ]:
paths = ['data/static0-0-3-preproc0.zip']

dataset_fn = 'nnsysident.datasets.mouse_loaders.static_loaders'
dataset_config = dict(
    paths=paths,
    batch_size=64,
    neuron_n=1000,
    neuron_base_seed=1,
    image_n=4000,
    image_base_seed=1

)

dataset_config_auto = dict()

# Add the Model

In [ ]:
model_fn = 'nnsysident.models.models.se2d_fullgaussian2d'
model_config = {
}

model_config_auto = dict(
    gamma_input={"type": "range", "bounds": [1e-2, 1e3], "log_scale": True},
    gamma_readout={"type": "range", "bounds": [1e-6, 1e-1], "log_scale": True},
)

# Add the Trainer

In [ ]:
trainer_fn = 'nnsysident.training.trainers.standard_trainer'
trainer_config = dict()
trainer_config_auto = dict(
              )

# Run Bayesian Search

In [ ]:
# Starting the bayesian Search:
# The number of total trials can be varied, but 200 trials is usually sufficient

In [ ]:
autobayes = Bayesian(dataset_fn, dataset_config, dataset_config_auto,
              model_fn, model_config, model_config_auto,
              trainer_fn, trainer_config, trainer_config_auto, architect="kklurz", trained_model_table='nnsysident.tables.bayesian.TrainedModelBayesian', total_trials=1)
 
best_parameters, _, _, _ = autobayes.run()